In [1]:
!pip install gekko

In [17]:
from gekko import GEKKO
import numpy as np
import random
from baconshor import *

In [21]:
# rep code decoder

# settings ----
d=3
num_qubits =d**2
num_stabilizers =(d - 1)**2
# p = 0.1

# ps = {} # error probs
# for i in range(num_qubits):
#     ps[i] = p # constant prob of error for now


grid = create_grid(3)
p = 0.1
import random
list1 = list(range(9))
#where to add y errors
error_list = []
for i in range(len(list1)):
    x = random.random()
    if x < p:
        error_list.append(list1[i]+1)
# print(errorlist)
add_y_error(grid,error_list)

Print(grid)


total = []
for i in range(d-1):
    listrow=[]
    listcol=[]
    for q in range(d):
        listrow.append(d*i +q)
        listrow.append((d)*(i+1) + q)
        # listcol.append((d*q +i))
        listcol.append(d*q +i)
        listcol.append(d*q +(i+1))

    total.append(listrow)
    total.append(listcol) 
# print(total)
Cs = {}
for l in total:
    Cs[tuple(l)] = check_stabilizer(grid, l)

# Cs = { # stabilizer check values, these are the measured stabilizer values
#     (0,1,2,3,4,5): 1, 
#     (3,4,5,6,7,8): 1,
#     (0,1,3,4,6,7): -1,
#     (1,2,4,5,7,8): -1,
# }
# The above stabilizers indicate Z0Z1 = -1 and Z1Z2 = 1, so there's probably an error on the 0th qubit
# Let's see if the MLE decoder can figure this out

# set up gekko ----
m = GEKKO(remote=False)
m.options.SOLVER = 1  # APOPT is an MINLP solver
# optional solver settings with APOPT
m.solver_options = [
    "minlp_maximum_iterations 500",  # minlp iterations with integer solution
    "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
    "minlp_as_nlp 0",  # nlp sub-problem max iterations
    "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
    "minlp_branch_method 1",  # maximum deviation from whole number
    "minlp_integer_tol 0.05",  # covergence tolerance
    "minlp_gap_tol 0.01",
]


# set up variables ---
Es = {}
for i in range(num_qubits):
    Es[i] = m.Var(
        value=random.randint(0,1), lb=0, ub=1, integer=True
    )

Ks = {}
for k in Cs.keys():
    Ks[k] = m.Var(
        value=random.randint(0,1), lb=0, integer=True
    )


# Objective ---
m.Obj(
    -m.sum(
        [
            np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
            for j in range(num_qubits)
        ]
    )
)


# Constraints ---

for key, val in Cs.items():
    i, j,k,l,o,n = key
    # m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)
    m.Equation(m.sum([Es[i] for i in key]) - 2 * Ks[key] == (1 - val)/2) #automatically sums over all the qubits in the stabilizer key

m.solve()

print("Solution (0: no error, 1: error): ", Es)

1 0 0 
0 0 0 
0 0 0 
 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            5
   Constants    :            0
   Variables    :           27
   Intermediates:            0
   Connections  :           38
   Equations    :           14
   Residuals    :           14
 
 Number of state variables:             27
 Number of total equations: -           18
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              9
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    3 Obj:  3.15E+00 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    1 Dpth:    1 

In [ ]:
#OUTPUT of decoder in the grid format to compare
#more general d
#make plot p vs percent decoder was right - run each p like 1000 times 
#fix ssh key